In [3]:
# Set random seed for reproducibility
import numpy as np
np.random.seed(42)

This notebook is merged from multiple notebooks created while working on this task. Each numbered section corresponds to one of the original notebooks.

# Classification and Regression of Artificial Data

This project involves working with an artificially generated dataset consisting of 2000 samples, 400 input variables, and two output variables: a discrete 'Class' variable (0 or 1) and a continuous 'Output' variable. The goal is to build ML models predicting these output variables based on the input features and analyze the dependencies discovered.

## 0. Testing the models

In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, r2_score

def evaluate_on_validation_data(validation_file_path):
    """Process a validation_data.csv file with the same structure as the training data
    and compute the classification accuracies and R^2 values for both baseline and best models.
    
    Args:
        validation_file_path: Path to the validation CSV file
        
    Returns:
        dict: Dictionary with all computed metrics
    """
    # Load validation data
    validation_df = pd.read_csv(validation_file_path, sep=';')
    X_val = validation_df.drop(['Class', 'Output'], axis=1)
    y_val_class = validation_df['Class']
    y_val_output = validation_df['Output']
    
    # Load and prepare training data (needed to fit models)
    train_df = pd.read_csv('data.csv', sep=';')
    X_train = train_df.drop(['Class', 'Output'], axis=1)
    y_train_class = train_df['Class']
    y_train_output = train_df['Output']
    
    # Classification baseline model (Logistic Regression)
    baseline_clf = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')
    baseline_clf.fit(X_train, y_train_class)
    baseline_clf_accuracy = accuracy_score(y_val_class, baseline_clf.predict(X_val))
    
    # Best classification model (Random Forest with RFE)
    # Create and fit RFE
    best_feature_count = 16  # From our optimization
    optimal_rfe = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42), 
                      n_features_to_select=best_feature_count, step=10)
    optimal_rfe.fit(X_train, y_train_class)
    X_train_rfe = optimal_rfe.transform(X_train)
    X_val_rfe = optimal_rfe.transform(X_val)
    
    # Best RF parameters from grid search
    best_rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=30,
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=True,
        random_state=42
    )
    best_rf.fit(X_train_rfe, y_train_class)
    best_clf_accuracy = accuracy_score(y_val_class, best_rf.predict(X_val_rfe))
    
    # Regression baseline model (Linear Regression)
    baseline_reg = LinearRegression()
    baseline_reg.fit(X_train, y_train_output)
    baseline_reg_r2 = r2_score(y_val_output, baseline_reg.predict(X_val))
    
    # Best regression model (Lasso with selected features)
    # Use Lasso to select features
    feature_selector = Lasso(alpha=0.1, max_iter=10000)
    feature_selector.fit(X_train, y_train_output)
    
    # Get non-zero coefficients
    selected_features = X_train.columns[feature_selector.coef_ != 0].tolist()
    X_train_lasso = X_train[selected_features]
    X_val_lasso = X_val[selected_features]
    
    # Best Lasso parameters from tuning
    best_lasso = Lasso(alpha=0.005, max_iter=10000)
    best_lasso.fit(X_train_lasso, y_train_output)
    best_reg_r2 = r2_score(y_val_output, best_lasso.predict(X_val_lasso))
    
    # Print results
    print("\nValidation Results:")
    print("-" * 40)
    print(f"Baseline Classification Accuracy: {baseline_clf_accuracy:.4f} ({baseline_clf_accuracy*100:.2f}%)")
    print(f"Best Classification Accuracy:     {best_clf_accuracy:.4f} ({best_clf_accuracy*100:.2f}%)")
    print(f"Baseline Regression R²:          {baseline_reg_r2:.4f}")
    print(f"Best Regression R²:              {best_reg_r2:.4f}")
    
    return {
        'baseline_classification_accuracy': baseline_clf_accuracy,
        'best_classification_accuracy': best_clf_accuracy,
        'baseline_regression_r2': baseline_reg_r2,
        'best_regression_r2': best_reg_r2
    }

# Example usage:
# results = evaluate_on_validation_data('validation_data.csv')

## 1. Data exploration

In [5]:
# Import libraries for data exploration
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [6]:
# Load the dataset
df = pd.read_csv('data.csv', sep=';')

# Display the first few rows
df.head()

,Class,Output,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,...,Input391,Input392,Input393,Input394,Input395,Input396,Input397,Input398,Input399,Input400
0,0,0.800586,-0.002583,2.184037,-0.322008,1.621241,1.192444,-0.278356,-0.207366,0.735689,...,-2.140861,1.187660,0.345238,-0.844885,0.580007,-2.605781,-0.299471,0.711487,-0.753316,0.728763
1,0,2.168475,0.668637,1.373933,-0.476868,-0.724704,0.031162,-1.845921,0.784890,1.508526,...,-1.286120,-0.900044,-0.500399,-0.126421,-0.632233,-2.557419,0.056044,0.634774,-0.259835,0.106390
2,1,-1.210777,-0.681438,-0.544753,0.441346,-0.019906,-0.192135,-0.162510,-0.998777,0.686472,...,-0.391605,-0.190147,0.793746,-0.812737,-0.068228,-0.313143,2.564096,0.848355,0.180556,-1.525615
3,1,0.505678,-0.497957,0.720712,0.149120,0.019251,1.377850,0.981337,-0.846813,0.036790,...,-0.176734,-0.947351,-0.888601,1.509450,-0.501929,-0.554909,-0.104051,0.442150,-0.056644,1.447267
4,1,-10.281033,-1.178544,0.176941,1.112202,1.234189,0.999451,-0.773329,-0.811075,1.550537,...,-0.181325,0.198960,-0.697497,-0.836371,1.652071,0.974292,1.584071,-0.202352,1.362426,1.023857


In [8]:
# Comprehensive data exploration
print("Dataset shape:", df.shape)

# Check class distribution
print("\nClass distribution:")
class_counts = df['Class'].value_counts()
print(class_counts)
print(f"Class 0: {class_counts[0]/len(df)*100:.2f}%")
print(f"Class 1: {class_counts[1]/len(df)*100:.2f}%")

# Continuous output variable statistics
print("\nOutput variable statistics:")
print(df['Output'].describe())

# Check missing values
print("\nMissing values:")
print(df.isnull().sum().sum())

# Separate features and targets
X = df.drop(['Class', 'Output'], axis=1)  # Use all Input variables as predictors
y_class = df['Class']  # Classification target
y_output = df['Output']  # Regression target

# Feature information
print(f"\nFeatures shape: {X.shape}")
print(f"Number of predictor variables: {X.shape[1]}")

Dataset shape: (2000, 402)

Class distribution:
Class
1    1013
0     987
Name: count, dtype: int64
Class 0: 49.35%
Class 1: 50.65%

Output variable statistics:
count    2000.000000
mean        0.106647
std         3.562855
min       -12.384019
25%        -2.310265
50%         0.140658
75%         2.435991
max        11.676146
Name: Output, dtype: float64

Missing values:
0

Features shape: (2000, 400)
Number of predictor variables: 400


## 2. Baseline Classification Model

We'll implement a logistic regression model as our baseline for the classification task, using all 400 input variables. We'll evaluate its performance on the training data and use cross-validation to estimate its ability to generalize.

In [10]:
# Build Logistic Regression Model
logistic_model = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')

# Fit the model on the training data
logistic_model.fit(X, y_class)

# Make predictions on training data
y_train_pred = logistic_model.predict(X)
y_train_pred_proba = logistic_model.predict_proba(X)[:, 1]

# Calculate training accuracy
train_accuracy = accuracy_score(y_class, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")

Training Accuracy: 0.7100 (71.00%)


In [11]:
# Use 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(logistic_model, X, y_class, cv=kf, scoring='accuracy')
cv_scores_mean = cv_scores.mean()

# Compare training vs cross-validation performance
print(f"\nPerformance Comparison:")
print(f"Training Accuracy:     {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"CV Mean Accuracy:      {cv_scores_mean:.4f} ({cv_scores_mean*100:.2f}%)")
print(f"Difference:            {train_accuracy - cv_scores.mean():.4f}")


Performance Comparison:
Training Accuracy:     0.7100 (71.00%)
CV Mean Accuracy:      0.5170 (51.70%)
Difference:            0.1930


### Baseline Classification Model Analysis

The logistic regression model trained on all 400 features shows signs of overfitting. The large gap between training accuracy and cross-validation accuracy indicates the model is not generalizing well to unseen data. This suggests that many of the 400 features might be noise rather than signal for the classification task, and dimensionality reduction or feature selection could improve performance.

## 3. Advanced classification model

In [12]:
# Model evaluation framework for classification
def evaluate_classification_model(model, X, y, cv_folds=5, model_name="Model"):
    """Comprehensive model evaluation function for classification"""
    # K-Fold for cross-validation
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    # Cross-validation scores
    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    
    # Fit model for training accuracy
    model.fit(X, y)
    train_pred = model.predict(X)
    train_accuracy = accuracy_score(y, train_pred)
    
    # Results
    results = {
        'model_name': model_name,
        'train_accuracy': train_accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_scores': cv_scores,
        'overfitting': train_accuracy - cv_scores.mean()
    }
    
    print(f"Model: {model_name}")
    print(f"Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
    print(f"CV Mean Accuracy:  {cv_scores.mean():.4f} ({cv_scores.mean()*100:.2f}%)")
    print(f"Overfitting:       {train_accuracy - cv_scores.mean():.4f}")
        
    return results

### Model Evaluation Summary

After exploring several approaches, here are the performance results of different models sorted by cross-validation accuracy:

| Model | CV Accuracy | Improvement |
|-------|------------|-------------|
| Baseline Logistic Regression | 52.4% | Baseline |
| Logistic Regression + SelectKBest | 61.1% | +8.7% |
| SVC (RBF) + SelectKBest | 67.5% | +15.1% |
| Random Forest | 66.1% | +13.7% |
| Random Forest + SelectKBest | 73.6% | +21.2% |
| Optimized Random Forest + SelectKBest | 74.9% | +22.5% |
| Random Forest + PCA + Lasso | 65.3% | +12.9% |
| **Random Forest + RFE** | **78.8%** | **+26.4%** |

The most promising approach uses Random Forest with Recursive Feature Elimination (RFE). Let's optimize this model further.

In [ ]:
# Find optimal number of features for RFE

# Test a range of feature counts
feature_range = [14, 16, 18, 20, 22, 24, 26]
rfe_results = {}

for n_features in feature_range:
    print(f"\nTesting RFE with {n_features} features...")
    
    # Create RFE with specified number of features
    rfe_test = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42), 
                  n_features_to_select=n_features, step=10, verbose=0)
    X_rfe_test = rfe_test.fit_transform(X, y_class)
    
    # Create and evaluate model with default parameters
    rf_test = RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    )
    
    # Use cross-validation for more reliable results
    cv_results = evaluate_classification_model(rf_test, X_rfe_test, y_class, model_name=f"RF + RFE ({n_features} features)")
    rfe_results[n_features] = cv_results

# Find best feature count
feature_counts = list(rfe_results.keys())
cv_means = [rfe_results[n]['cv_mean'] for n in feature_counts]
best_feature_count = feature_counts[np.argmax(cv_means)]
best_cv_mean = max(cv_means)

print(f"\nBest feature count: {best_feature_count} with CV accuracy: {best_cv_mean:.4f} ({best_cv_mean*100:.2f}%)")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(feature_counts, [score*100 for score in cv_means], marker='o', linestyle='-')
plt.axvline(x=best_feature_count, color='r', linestyle='--', label=f'Best: {best_feature_count} features')
plt.xlabel('Number of Features')
plt.ylabel('Cross-Validation Accuracy (%)')
plt.title('RFE Feature Count Optimization')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Use the best feature count for further optimization
optimal_rfe = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42), 
                  n_features_to_select=best_feature_count, step=10, verbose=0)
X_rfe_optimal = optimal_rfe.fit_transform(X, y_class)

# Get feature names
optimal_feature_indices = np.where(optimal_rfe.ranking_ == 1)[0]
optimal_feature_names = X.columns[optimal_feature_indices]
print(f"\nOptimal features selected: {optimal_feature_names.tolist()}")


Testing RFE with 10 features...
Model: RF + RFE (10 features)
Training Accuracy: 1.0000 (100.00%)
CV Mean Accuracy:  0.7950 (79.50%)
Overfitting:       0.2050

Testing RFE with 12 features...
Model: RF + RFE (10 features)
Training Accuracy: 1.0000 (100.00%)
CV Mean Accuracy:  0.7950 (79.50%)
Overfitting:       0.2050

Testing RFE with 12 features...
Model: RF + RFE (12 features)
Training Accuracy: 1.0000 (100.00%)
CV Mean Accuracy:  0.8100 (81.00%)
Overfitting:       0.1900

Testing RFE with 14 features...
Model: RF + RFE (12 features)
Training Accuracy: 1.0000 (100.00%)
CV Mean Accuracy:  0.8100 (81.00%)
Overfitting:       0.1900

Testing RFE with 14 features...
Model: RF + RFE (14 features)
Training Accuracy: 1.0000 (100.00%)
CV Mean Accuracy:  0.8145 (81.45%)
Overfitting:       0.1855

Testing RFE with 16 features...
Model: RF + RFE (14 features)
Training Accuracy: 1.0000 (100.00%)
CV Mean Accuracy:  0.8145 (81.45%)
Overfitting:       0.1855

Testing RFE with 16 features...
Model: 

In [ ]:
# Hyperparameter optimization for Random Forest

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Create Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Set up grid search with cross-validation
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Fit grid search
grid_search.fit(X_rfe_optimal, y_class)

# Print best parameters and score
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f} ({grid_search.best_score_*100:.2f}%)")

# Get best model
best_rf = grid_search.best_estimator_

In [ ]:
# Final evaluation of best model with rigorous cross-validation
evaluate_classification_model(best_rf, X_rfe_optimal, y_class, cv_folds=10, model_name="Fully Optimized RF + RFE")

### Advanced Classification Model Analysis

Our optimized Random Forest classifier with Recursive Feature Elimination (RFE) achieved significant improvements over the baseline model:

1. **Feature Selection**: We identified the optimal number of features (16) out of 400, dramatically reducing dimensionality while improving performance.

2. **Model Selection**: Random Forest outperformed other classifiers for this task.

3. **Hyperparameter Optimization**: Grid search identified the optimal parameters, further improving performance.

**Performance Improvement**: The final model achieved approximately 80% accuracy in cross-validation, compared to 52% for the baseline model, representing a substantial 28 percentage point improvement.


While there is still some overfitting (difference between training and CV accuracy), this is expected with Random Forest models, and the high cross-validation accuracy indicates good generalization ability.

## 4. Baseline regression model

In [ ]:
# Build Linear Regression Model
linear_model = LinearRegression()

# Fit the model on the training data
linear_model.fit(X, y_output)

# Make predictions on training data
y_pred_train = linear_model.predict(X)

# Calculate training R^2 and RMSE
train_mse = mean_squared_error(y_output, y_pred_train)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_output, y_pred_train)

print(f"Training R² Score: {train_r2:.4f}")
print(f"Training RMSE: {train_rmse:.4f}")

In [ ]:
# Use 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Calculate cross-validation scores for different metrics
cv_r2_scores = cross_val_score(linear_model, X, y_output, cv=kf, scoring='r2')
cv_neg_mse_scores = cross_val_score(linear_model, X, y_output, cv=kf, scoring='neg_mean_squared_error')

# Convert negative scores to positive
cv_mse_scores = -cv_neg_mse_scores
cv_rmse_scores = np.sqrt(cv_mse_scores)

# Calculate mean
cv_r2_mean = cv_r2_scores.mean()
cv_rmse_mean = cv_rmse_scores.mean()

print("\nPerformance Comparison:")
print(f"Training R²: {train_r2:.4f}, Training RMSE: {train_rmse:.4f}")
print(f"Mean CV R²: {cv_r2_scores.mean():.4f}, Mean CV RMSE: {cv_rmse_scores.mean():.4f}")
print(f"Difference in R²: {train_r2 - cv_r2_mean:.4f}, Difference in RMSE: {train_rmse - cv_rmse_mean:.4f}")

### Baseline Regression Model Analysis

The linear regression model with all 400 features shows significant overfitting:

1. **High Training R²**: The model fits the training data very well, with an R² score near 1.0.

2. **Poor Cross-Validation R²**: The much lower cross-validation R² indicates the model doesn't generalize well.

3. **Large RMSE Difference**: The gap between training and cross-validation RMSE further confirms poor generalization.

This suggests that, similar to the classification task, many of the 400 features might be noise rather than signal for predicting the 'Output' variable. Feature selection or regularization could help improve model performance.

## 5. Advanced regression model

In [ ]:
# Define evaluation function for regression models
def evaluate_regression_model(model, X, y):
    """Evaluate regression model with training metrics and cross-validation"""
    # 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Get training metrics
    model.fit(X, y)
    y_pred_train = model.predict(X)
    train_mse = mean_squared_error(y, y_pred_train)
    train_rmse = np.sqrt(train_mse)
    train_r2 = r2_score(y, y_pred_train)
    
    # Cross-validation scores
    cv_r2_scores = cross_val_score(model, X, y, cv=kf, scoring='r2')
    cv_neg_mse_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
    
    # Convert negative scores to positive
    cv_mse_scores = -cv_neg_mse_scores
    cv_rmse_scores = np.sqrt(cv_mse_scores)
    
    # Calculate mean
    cv_r2_mean = cv_r2_scores.mean()
    cv_rmse_mean = cv_rmse_scores.mean()
    
    print(f"Training R²: {train_r2:.4f}, Training RMSE: {train_rmse:.4f}")
    print(f"Mean CV R²: {cv_r2_mean:.4f}, Mean CV RMSE: {cv_rmse_mean:.4f}")
    
    return {
        'train_r2': train_r2,
        'train_rmse': train_rmse,
        'cv_r2': cv_r2_mean,
        'cv_rmse': cv_rmse_mean
    }

### Model Evaluation Results

After testing various regression models with different preprocessing techniques, here are the results sorted by cross-validation R² score (higher is better):

| Model | CV R² | CV RMSE |
|-------|-------|--------|
| Lasso(alpha=0.1) | 0.4861 | 2.5445 |
| LinearRegression_RFE(n=20) | 0.4859 | 2.5433 |
| LinearRegression_RFE(n=30) | 0.4708 | 2.5799 |
| LinearRegression_SelectKBest(k=30) | 0.4635 | 2.6003 |
| LinearRegression_SelectKBest(k=50) | 0.4487 | 2.6335 |
| LinearRegression_SelectKBest(k=20) | 0.4396 | 2.6570 |
| Ridge(alpha=100.0)_SelectKBest(k=20) | 0.4395 | 2.6574 |
| Ridge(alpha=100.0) | 0.3979 | 2.7509 |
| LinearRegression_SelectKBest(k=10) | 0.3760 | 2.8051 |
| RandomForestRegressor_SelectKBest(k=20) | 0.3549 | 2.8531 |
| LinearRegression_PCA(n=50) | 0.2842 | 3.0068 |
| LinearRegression_PCA(n=30) | 0.2777 | 3.0076 |
| LinearRegression_PCA(n=20) | 0.2736 | 3.0283 |
| RandomForestRegressor | 0.2719 | 3.0314 |
| **Baseline Linear Regression** | **0.3750** | **2.8024** |

The best model was **Lasso Regression with alpha=0.1**, achieving a cross-validation R² of 0.4861, which represents an improvement of 0.1111 (11.11 percentage points) over the baseline model. Let's optimize this model further.

In [ ]:
# Use Lasso to select features
base_alpha = 0.1

# Use Lasso to select features
feature_selector = Lasso(alpha=base_alpha, max_iter=10000)
feature_selector.fit(X, y_output)

# Get non-zero coefficients
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': feature_selector.coef_
})

# Sort by absolute coefficient values
coef_df['Abs_Coefficient'] = np.abs(coef_df['Coefficient'])
coef_df = coef_df.sort_values('Abs_Coefficient', ascending=False)

# Filter features with non-zero coefficients
non_zero_coef = coef_df[coef_df['Coefficient'] != 0].copy()

# Create dataset with selected features only
selected_features = non_zero_coef['Feature'].tolist()
X_lasso_selected = X[selected_features]

print(f"Original feature count: {X.shape[1]}")
print(f"Selected feature count: {X_lasso_selected.shape[1]}")

# Display top 10 features by coefficient magnitude
print("\nTop 10 most important features:")
print(non_zero_coef.head(10))

# Plot top 20 feature coefficients
top_features = non_zero_coef.head(20).sort_values('Coefficient')
plt.figure(figsize=(12, 8))
plt.barh(top_features['Feature'], top_features['Coefficient'])
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Top 20 Feature Coefficients')
plt.grid(alpha=0.3)
plt.show()

# Evaluate base Lasso with selected features
lasso_base = Lasso(alpha=base_alpha, max_iter=10000)
lasso_results = evaluate_regression_model(lasso_base, X_lasso_selected, y_output)

In [ ]:
# Define a range of alpha values to test
alphas = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3]

# Create a dictionary to store results
alpha_results = {}

# Test each alpha value
for alpha in alphas:
    lasso_model = Lasso(alpha=alpha, max_iter=10000)
    print(f"\nTesting Lasso with alpha={alpha}")
    results = evaluate_regression_model(lasso_model, X_lasso_selected, y_output)
    alpha_results[alpha] = results['cv_r2']

# Find the best alpha
best_alpha = max(alpha_results, key=alpha_results.get)
best_score = alpha_results[best_alpha]

print(f"\nBest alpha: {best_alpha} with CV R²: {best_score:.4f}")

# Plot alpha vs R²
plt.figure(figsize=(10, 6))
plt.semilogx(list(alpha_results.keys()), list(alpha_results.values()), marker='o')
plt.axvline(x=best_alpha, color='r', linestyle='--', label=f'Best alpha: {best_alpha}')
plt.xlabel('Alpha (log scale)')
plt.ylabel('Cross-Validation R²')
plt.title('Lasso Alpha Parameter Tuning')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Save the best Lasso model
best_lasso = Lasso(alpha=best_alpha, max_iter=10000)
best_lasso_results = evaluate_regression_model(best_lasso, X_lasso_selected, y_output)

### Advanced Regression Model Analysis

Our optimized Lasso regression model achieved significant improvements over the baseline:

1. **Feature Selection**: Lasso automatically identified the most important features from the original 400, reducing dimensionality while improving performance.

2. **Regularization**: By penalizing large coefficients, Lasso helped prevent overfitting and improved generalization.

3. **Parameter Optimization**: Tuning the alpha parameter further improved performance.

4. **Performance Improvement**: The final model achieved a cross-validation R² of approximately 0.50, compared to 0.38 for the baseline model.

While there is still room for improvement, the advanced model shows substantially better generalization ability than the baseline linear regression model. I've tried several other techniques to further improve the model, such as adding polynomial features, but it did not give promising results.

## 6. Conclusions and Key Findings

### Overall Project Summary

In this project, we worked with an artificially generated dataset consisting of 2000 samples and 400 input variables to predict two output variables: a discrete 'Class' variable and a continuous 'Output' variable. Here are the key findings:

### Classification Task

1. **Baseline Performance**: Logistic Regression using all 400 features achieved a cross-validation accuracy of approximately 52%.

2. **Best Model**: Random Forest with Recursive Feature Elimination (RFE) selecting only 16 features achieved a cross-validation accuracy of approximately 80%.

3. **Improvement**: An absolute improvement of ~28 percentage points in accuracy.


### Regression Task

1. **Baseline Performance**: Linear Regression using all 400 features achieved a cross-validation R² of approximately 0.38.

2. **Best Model**: Lasso Regression with optimized alpha parameter achieved a cross-validation R² of approximately 0.50.

3. **Improvement**: An absolute improvement of ~12 percentage points in R².